In [1]:
"""
I had a idea from work like below for my final projects, I think I will doing solo but if anyone want to work together you are more than welcome to DM me. 

Background: My job is sale directors for steel companies, so we hitting the road really often, when we going out to visit customer, we will visit 3-4 customer at the same time. Everytime I need to find information like what is the sale volume to customer last month, what is the sizes, what is the average prices or previous month prices, I need to open my laptop, get to the ERP and found the information. 

My project: I will made use a local LLMs and Perplexity as my personal assistance. So I will make RAG to access the database, take out the information I need, example like the list of sizes we sold to customers. The material description can be change but can be also repeating multiple times, the diffirences is in the volume wise.


Interface: I think I will use Gradio to do chat interface, then deploy to Hughing Face with address that only me can access 

Extra Credit: It is already a structured data to begin with hahaha. I am thinking about routing agents as tools call.
There will be 3 tools calling:
1/Use PandasQueryEngines to get information of averages prices etc...
2/Use Tools to drawing a charts using data from PandasQuery Engines results
3/Using tools to passed the Queries to Perplexity when I asking something like where is the company, what information it hads. Meaning outside of the RAG Database and most up-to-date data. 

The table will have 8 columns:
Customer name, Item code, Description, Quantity, Unit, Price per unit, Total value, Billing date. 

This 8 columns enough data for me. From customer name, if I want any other information, Perplexity can help me. 

I choose to reduce the complexity of the data in order to 1 single client data including a very few columns but having many many rows instead, this should fit to my need, no need to get a really complex multiple relationship database. 

FYI: The description will be in Vietnamese so Local LLM need to able to handle multi-language.
"""

'\nI had a idea from work like below for my final projects, I think I will doing solo but if anyone want to work together you are more than welcome to DM me. \n\nBackground: My job is sale directors for steel companies, so we hitting the road really often, when we going out to visit customer, we will visit 3-4 customer at the same time. Everytime I need to find information like what is the sale volume to customer last month, what is the sizes, what is the average prices or previous month prices, I need to open my laptop, get to the ERP and found the information. \n\nMy project: I will made use a local LLMs and Perplexity as my personal assistance. So I will make RAG to access the database, take out the information I need, example like the list of sizes we sold to customers. The material description can be change but can be also repeating multiple times, the diffirences is in the volume wise.\n\n\nInterface: I think I will use Gradio to do chat interface, then deploy to Hughing Face wit

In [26]:
"""
!pip install -q pandas
!pip install -q openai
!pip install -q sentence-transformers
!pip install -q llama-index-program-openai
!pip install -q llama-index-llms-openai
!pip install -q llama-index-experimental
!pip install -q llama-index-embeddings-huggingface
!pip install -q llama-index-embeddings-fastembed
!pip install -q fastembed
!pip install -q xlrd>=2.0.1
"""

'\n!pip install -q pandas\n!pip install -q openai\n!pip install -q sentence-transformers\n!pip install -q llama-index-program-openai\n!pip install -q llama-index-llms-openai\n!pip install -q llama-index-experimental\n!pip install -q llama-index-embeddings-huggingface\n!pip install -q llama-index-embeddings-fastembed\n!pip install -q fastembed\n!pip install -q xlrd>=2.0.1\n'

In [3]:
import os
import ollama
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core import PromptTemplate



In [4]:
"""
Loading all the API from .env file
"""
dotenv_path = find_dotenv()  # Searches up the directory tree
#print(f"Loading dotenv from: {dotenv_path}")
load_dotenv(dotenv_path)
openai_api_key = os.getenv('OPENAI_API_KEY')
hf_key = os.getenv('HF_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key

In [5]:
"""
Setuping Ollama by install Ollama, 
Then ollama pull llama3.1
"""

Ollama_MODEL = "llama3.1"  # Specifies the Ollama model to use.
Ollama_LLM = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
"""
response = Ollama_LLM.chat.completions.create(
    model=Ollama_MODEL,
    messages=[
        {"role": "user", "content": "Who are you? What is your name, version and Parameter?  What is your strength? Do you support Vietnamese language"}
    ]
)
print(response.choices[0].message.content)
"""

'\nresponse = Ollama_LLM.chat.completions.create(\n    model=Ollama_MODEL,\n    messages=[\n        {"role": "user", "content": "Who are you? What is your name, version and Parameter?  What is your strength? Do you support Vietnamese language"}\n    ]\n)\nprint(response.choices[0].message.content)\n'

In [6]:
"""
Setuping Deepseek (from Ollama) by install Ollama
Then ollama pull deepseek-r1:8b
"""

Deepseek_MODEL = "deepseek-r1:8b"  # Specifies the Ollama model to use.
Deepseek_LLM = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

response = Deepseek_LLM.chat.completions.create(
    model=Deepseek_MODEL,
    messages=[
        {"role": "user", "content": "Who are you? What is your name, version and Parameter?  What is your strength? Do you support Vietnamese language"}
    ]
)
print(response.choices[0].message.content) 


INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
<think>
I'm DeepSeek-R1-Lite-Preview, an AI assistant created exclusively by the Chinese Company DeepSeek. I'll do my best to help you.
My strengths include providing detailed explanations on various topics, helping with research, and offering creative solutions. I also specialize in multi-language support, including English, Chinese, Vietnamese, and more. I'm here to help you with a wide range of tasks, from answering questions to aiding with storytelling. Let's get started!
</think>

I'm DeepSeek-R1-Lite-Preview, an AI assistant created exclusively by the Chinese Company DeepSeek. I'll do my best to help you.

My strengths include providing detailed explanations on various topics, helping with research, and offering creative solutions. I also specialize in multi-language support, including English, Chinese, Vietname

In [7]:
def load_excel_files(folder_path):
    """Loads multiple .xls files from a folder into a single pandas DataFrame.

    Args:
        folder_path: The path to the folder containing the .xls files.

    Returns:
        A pandas DataFrame containing the combined data from all .xls files,
        or None if no .xls files are found or an error occurs.
    """
    all_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".xls"):
            file_path = os.path.join(folder_path, filename)
            try:
                df = pd.read_excel(file_path)
                all_data.append(df)
            except Exception as e:
                print(f"Error reading file {filename}: {e}")
                # Handle the error appropriately (e.g., skip the file, raise an exception)
                return None

    if not all_data:
        print("No .xls files found in the specified folder.")
        return None

    try:
      combined_df = pd.concat(all_data, ignore_index=True)
      return combined_df
    except Exception as e:
      print(f"Error combining dataframes: {e}")
      return None
"""
# Usage:
folder_path = "Sale_Data"  # Replace with the actual path to your folder
df = load_excel_files(folder_path)

#Debug:
if df is not None:
    print(df.head())  # Print the first few rows of the combined DataFrame
"""

'\n# Usage:\nfolder_path = "Sale_Data"  # Replace with the actual path to your folder\ndf = load_excel_files(folder_path)\n\n#Debug:\nif df is not None:\n    print(df.head())  # Print the first few rows of the combined DataFrame\n'

In [8]:
# initialize empty df and load Sale_Data
df = load_excel_files("Sale_Data")
df.head()

,Company_Name,Item_Code,Item_Details,Quantity,Unit,Value,Currency,Billing Date
0,Công Ty TNHH Một Thành Viên Quang Min,50000304.0,HR 2.3x250x4C,8470.0,KG,116886000.0,VND,2019-02-16
1,Công Ty TNHH Một Thành Viên Quang Min,50000304.0,HR 2.3x45x3C,1135.0,KG,15663000.0,VND,2019-02-16
2,Công Ty TNHH Một Thành Viên Quang Min,50000363.0,HR 2.0x1114x3000x220T,11110.0,KG,154429000.0,VND,2019-02-15
3,Công Ty TNHH Một Thành Viên Quang Min,50000030.0,CRFH 1.5x170x5Coil,3200.0,KG,47040000.0,VND,2019-02-20
4,Công Ty TNHH Một Thành Viên Quang Min,50000030.0,CRFH 1.5x193x5Coil,2304.0,KG,33868800.0,VND,2019-02-20


In [ ]:
# Get the unique payer names
unique_payers = df['Company_Name'].unique()

# Display the unique payer names
#unique_payers

In [ ]:
#query_engine = PandasQueryEngine(df=df, verbose=True) #OpenAI
query_engine = PandasQueryEngine(df=df, llm=Deepseek_LLM, verbose=True)
#query_engine = PandasQueryEngine(df=df, llm=Ollama_LLM, verbose=True)
"""
Initializes a PandasQueryEngine to query a pandas DataFrame using a language model.

This query engine allows you to ask questions about the data in the DataFrame using natural language.
The `llm` argument specifies the language model to use for generating queries.
The `verbose=True` argument enables detailed logging of the query execution process.
"""
prompts = query_engine.get_prompts()
#print(prompts["pandas_prompt"].template)

In [ ]:
new_prompt = PromptTemplate(
    """\
You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
The dataframe contains data about sale data. 
It have 8 columns: 
Company_Name is the full name of the buyer
Item_Code is the ERP 
Item_Details is the item information including the thickness, width, length of the steel
Quantity is the purchasing quantity,
Unit is the unit of purchasing quantity,
Value is the purchasing value, 
Currency is the unit of purchasing value, 
Billing Date is the date that issues the billing or invoice					

This is the result of `print(df.head())`: {df_str}

Follow these instructions: {instruction_str}
Query: {query_str}

Return the answer from the dataframe with a natural language explanation of the answer.
"Generate *only* Pandas code to answer the question. Do not include any surrounding text or keywords, such as 'python'."
Expression: """
)

query_engine.update_prompts({"pandas_prompt": new_prompt})
prompts = query_engine.get_prompts()
#print(prompts["pandas_prompt"].template)

In [ ]:
# Replace predict with create
def _predict(prompt, **kwargs):
    """Helper function to make predict calls using the Ollama API."""
    global Ollama_MODEL  # Access the global variable
    response = Ollama_LLM.chat.completions.create(
        model=Ollama_MODEL,
        messages=[
            {"role": "system", "content": "You are an pandas dataframe expert, user will gave you an query, you will respond with Python executable query code using Pandas. Follow user instruction step by step."},
            {"role": "user", "content": prompt.format(**kwargs)}
        ]
    )
    #print(prompts["pandas_prompt"].template)
    #print(prompt.format(**kwargs))
    return response.choices[0].message.content

# Monkey patch the predict method onto the Ollama_LLM object
Ollama_LLM.predict = _predict

In [ ]:
# Replace predict with create
def _predict(prompt, **kwargs):
    """Helper function to make predict calls using the Ollama API."""
    global Deepseek_MODEL  # Access the global variable
    response = Deepseek_LLM.chat.completions.create(
        model=Deepseek_MODEL,
        messages=[
            {"role": "system", "content": "You are an pandas dataframe expert, user will gave you an query, you will respond with Python executable query code using Pandas. Follow user instruction step by step."},
            {"role": "user", "content": prompt.format(**kwargs)}
        ]
    )
    #print(prompts["pandas_prompt"].template)
    #print(prompt.format(**kwargs))
    return response.choices[0].message.content

# Monkey patch the predict method onto the Ollama_LLM object
Deepseek_LLM.predict = _predict

In [13]:
'''
response = query_engine.query("What item details, quantiy and value did CÔNG TY TNHH ASABA VIET NAM MANUFACTURING company bought?")
print(str(response))
'''

'\nresponse = query_engine.query("What item details, quantiy and value did CÔNG TY TNHH ASABA VIET NAM MANUFACTURING company bought?")\nprint(str(response))\n'

In [14]:
'''
Pandas_intrusction = response.metadata['pandas_instruction_str']
Pandas_output = response.metadata['raw_pandas_output']
print(Pandas_intrusction)
print(Pandas_output)
'''

"\nPandas_intrusction = response.metadata['pandas_instruction_str']\nPandas_output = response.metadata['raw_pandas_output']\nprint(Pandas_intrusction)\nprint(Pandas_output)\n"

In [15]:
"""
Since Local LLM is not smart, so I made a loop to running LLM multiple times with same prompts
until it got the answer.
"""

def check_Query_output(prompt):
    Error_check = False
    Pandas_output = None  # Initialize Pandas_output

    while not Error_check:  # Corrected while loop condition
        Query_Answer = query_engine.query(prompt)
        print(Query_Answer)
        if Query_Answer.metadata and 'raw_pandas_output' in Query_Answer.metadata:
            Pandas_intrusction = Query_Answer.metadata['pandas_instruction_str']
            Pandas_output = Query_Answer.metadata['raw_pandas_output']
            #print(Pandas_output)
            if "error" in Pandas_output.lower():
                prompt = prompt + " " + Pandas_output + " " + Pandas_intrusction
                Error_check = False
            else:
                Error_check = True
        else:
            Pandas_output = str(Query_Answer)  # Return the string of the query answer.
            Error_check = True # exit the loop.

    return Pandas_output

In [16]:
'''
output = check_Query_output("What item details, quantiy and value did CÔNG TY TNHH ASABA VIET NAM MANUFACTURING company bought?")
print(output)
'''

'\noutput = check_Query_output("What item details, quantiy and value did CÔNG TY TNHH ASABA VIET NAM MANUFACTURING company bought?")\nprint(output)\n'

In [ ]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core import StorageContext,load_index_from_storage

# Import necessary classes from the llama_index package
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader, Document

# Read documents from the specified directory and load a specific document, "report.pdf".
documents = [Document(text=payer_name) for payer_name in df['Company_Name'].unique()]

# Create a VectorStoreIndex object from the documents. This will involve processing the documents
# and creating a vector representation for each of them, suitable for semantic searching.
embeddings = FastEmbedEmbedding()
index = VectorStoreIndex.from_documents(documents, embeddings=embeddings)
                                    
# Create a storage context
storage_context = StorageContext.from_defaults(vector_store=index.vector_store)

# Persist the storage context to disk
storage_context.persist()

# Convert the VectorStoreIndex object into a query engine. This query engine can be used to
# perform semantic searches on the index, matching natural language queries to the most relevant
# documents in the index.
query_engine_name = index.as_query_engine()

# Use the query engine to search for documents that are relevant to the query
# from the indexed documents based on the semantic understanding of the query.
#response = query_engine_name.query("What is company asaba full name?")

# Print the response obtained from the query. This will display the result of the semantic search,
# showing the information or documents that best match the query about the 2024 outlook.
#print(response)

In [ ]:
def get_company_name(query):
    response = Ollama_LLM.chat.completions.create(
        model=Ollama_MODEL,
        messages=[{"role": "user", "content": f"Extract the company name from this query: '{query}'. Return only the company name."}]
    )
    return response.choices[0].message.content
#get_company_name("What did company asaba buy?")

In [ ]:
def update_company_name(query_text, company, response):

    updated_query = query_text.lower().replace(company.lower(), response)
    return updated_query

In [ ]:
def update_query_with_company_name(query_text):

    #print(f"Original Query: {query_text}")

    company = get_company_name(query_text)
    #print(f"Extracted Company: {company}")

    response = query_engine_name.query(f"What is the company:'{company}' full name?")
    #print(f"Search Response: {response}")

    response_text = str(response)

    updated_query = update_company_name(query_text, company, response_text)
    #print(f"Updated Query: {updated_query}")

    return updated_query

In [ ]:
def process_query(query_text):
    updated_query = update_query_with_company_name(query_text)
    result = check_Query_output(updated_query)
    return result

In [ ]:
process_query("What item details, quantiy and value did asaba company bought?")

INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:openai._base_client:Retrying request to /chat/completions in 0.484374 seconds
Retrying request to /chat/completions in 0.484374 seconds
INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
<think>
Okay, so the user is asking about the item details, quantity, and value that a specific company bought. The company name given is "CÔNG TY TNHH ASABA VI

Traceback (most recent call last):
  File "C:\Users\Khoa-Mac-PC\anaconda3\envs\jupyter-ai\Lib\site-packages\llama_index\experimental\query_engine\pandas\output_parser.py", line 42, in default_output_processor
    tree = ast.parse(output)
           ^^^^^^^^^^^^^^^^^
  File "C:\Users\Khoa-Mac-PC\anaconda3\envs\jupyter-ai\Lib\ast.py", line 52, in parse
    return compile(source, filename, mode, flags,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<unknown>", line 8
    Once I have the filtered dataframe, I need to extract three columns: Item_Details, Quantity, and Value. So for each of those, I'll use df['Item_Details'], df['Quantity'], and df['Value'].
                                                                                                                                                                                            ^
SyntaxError: unterminated string literal (detected at line 8)


INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```


The final answer in executable Python code using Pandas:

```python
(df[df['Company_Name'] == "CÔNG Ty TNHH ASABA VIET NAM MANUFACTURING"]['Item_Details','Quantity','Value'])
```
```
> Pandas Output: There was an error running the output as Python code. Error message: name 'python' is not defined
There was an error running the output as Python code. Error message: name 'python' is not defined


Traceback (most recent call last):
  File "C:\Users\Khoa-Mac-PC\anaconda3\envs\jupyter-ai\Lib\site-packages\llama_index\experimental\query_engine\pandas\output_parser.py", line 44, in default_output_processor
    safe_exec(ast.unparse(module), {}, local_vars)  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Khoa-Mac-PC\anaconda3\envs\jupyter-ai\Lib\site-packages\llama_index\experimental\exec_utils.py", line 171, in safe_exec
    return exec(__source, _get_restricted_globals(__globals), __locals)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'python' is not defined


INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```


To address this specific question, I will provide you with a tailored pandas query that extracts the required information.

1. Filter the dataframe based on "Company_Name".
2. Select the "Item_Details", "Quantity", and "Value" columns.

The final expression is:

expression = (
    df[df['Company_Name'] == 'CÔNG Ty TNHH ASABA VIET NAM MANUFACTURING']
       .loc[:, ['Item_Details', 'Quantity', 'Value']]
)

This can be evaluated using:

eval(expression)

The error occurred because the code was not properly enclosed in parentheses when used with eval(). The corrected expression is provided below.
```
> Pandas Output: There was an error running the output as Python code. Error message: invalid syntax (<unknown>, line 1)
There was an error running the output as Python code. Error message: inval

...

Traceback (most recent call last):
  File "C:\Users\Khoa-Mac-PC\anaconda3\envs\jupyter-ai\Lib\site-packages\llama_index\experimental\query_engine\pandas\output_parser.py", line 42, in default_output_processor
    tree = ast.parse(output)
           ^^^^^^^^^^^^^^^^^
  File "C:\Users\Khoa-Mac-PC\anaconda3\envs\jupyter-ai\Lib\ast.py", line 52, in parse
    return compile(source, filename, mode, flags,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<unknown>", line 1
    **Answer:**
    ^^
SyntaxError: invalid syntax


INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```


The correct expression to use in Python for evaluating is:

(
    df[df['Company_Name'] == 'CÔNG Ty TNHH ASABA VIET NAM MANUFACTURING']
       .loc[:, ['Item_Details', 'Quantity', 'Value']]
)

This expression can be evaluated using eval().
```
> Pandas Output: There was an error running the output as Python code. Error message: invalid syntax (<unknown>, line 1)
There was an error running the output as Python code. Error message: invalid syntax (<unknown>, line 1)


Traceback (most recent call last):
  File "C:\Users\Khoa-Mac-PC\anaconda3\envs\jupyter-ai\Lib\site-packages\llama_index\experimental\query_engine\pandas\output_parser.py", line 42, in default_output_processor
    tree = ast.parse(output)
           ^^^^^^^^^^^^^^^^^
  File "C:\Users\Khoa-Mac-PC\anaconda3\envs\jupyter-ai\Lib\ast.py", line 52, in parse
    return compile(source, filename, mode, flags,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<unknown>", line 1
    The correct expression to use in Python for evaluating is:
        ^^^^^^^
SyntaxError: invalid syntax


INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```


The final expression is:

(
    df[df['Company_Name'] == "CÔNG Ty TNHH ASABA VIET NAM MANUFACTURING"]
       .loc[:, ['Item_Details', 'Quantity', 'Value']]
)

This code will:
1. Filter the dataframe for rows where Company_Name matches "CÔNG Ty TNHH ASABA VIET NAM MANUFACTURING"
2. Select the Item_Details, Quantity, and Value columns.
3. Return a DataFrame containing this information.

The expression can be evaluated using eval():

eval("(
    df[df['Company_Name'] == 'CÔNG Ty TNHH ASABA VIET NAM MANUFACTURING']
       .loc[:, ['Item_Details', 'Quantity', 'Value']]
)")

This returns a DataFrame with the specified details for the matching company.
```
> Pandas Output: There was an error running the output as Python code. Error message: unterminated string literal (detected at line 15) (<unkn

Traceback (most recent call last):
  File "C:\Users\Khoa-Mac-PC\anaconda3\envs\jupyter-ai\Lib\site-packages\llama_index\experimental\query_engine\pandas\output_parser.py", line 42, in default_output_processor
    tree = ast.parse(output)
           ^^^^^^^^^^^^^^^^^
  File "C:\Users\Khoa-Mac-PC\anaconda3\envs\jupyter-ai\Lib\ast.py", line 52, in parse
    return compile(source, filename, mode, flags,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<unknown>", line 15
    eval("(
         ^
SyntaxError: unterminated string literal (detected at line 15)


In [ ]:
import gradio as gr
iface = gr.ChatInterface(
    fn=process_query,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Enter your query here..."),
    title="Pandas Query Chat with Memory",
    description="Enter a query to interact with your pandas DataFrame. The chat remembers your conversation."
)

iface.launch()